# Setup

In [21]:
# Load required packages
library(tidyverse)
library(broom)

## Wrangle inputs and outputs from Snakemake

In [3]:
# Inputs
markers_blast_paths <- snakemake@input[1:8] # We want this to stay as list
DG_names_path <- snakemake@input[[9]] # Remaining inuts as character vectors
SG_names_path <- snakemake@input[[10]]
DG_genMap_path <- snakemake@input[[11]]
SG_genMap_path <- snakemake@input[[12]]
TrR_v5_chromosomes_path <- snakemake@input[[13]]
occ1_paf_path <- snakemake@input[[14]]
occ2_paf_path <- snakemake@input[[15]]
pall1_paf_path <- snakemake@input[[16]]
pall2_paf_path <- snakemake@input[[17]]

# Outputs
BLASThits_LGxSG_plot_path <- snakemake@output[[1]]
scaffoldLengths_path <- snakemake@output[[2]]
longestScaffolds_path <- snakemake@output[[3]]
chrom_mapping_path <- snakemake@output[[4]]
chrom_orientations_plot_path <- snakemake@output[[5]]
chrom_orientations_df_path <- snakemake@output[[6]]

## Functions

Load custom functions used throughout notebook

In [4]:
# Function to load results of BLASTING linkage markers against the haplotypes
load_marker_blast <- function(path){
  
    path_split <- str_split(basename(path), pattern = '_', simplify = TRUE)
    marker_pop <- path_split[2][1]
    sg_hap <- path_split[1][1]
    sg <- str_extract(sg_hap, pattern = 'occ|pall')
    hap <- str_extract(sg_hap, pattern = '1|2')

    # Column names
    cols <- c('qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 
              'qstart', 'qend', 'qlen', 'sstart', 'send', 'slen', 'evalue', 
              'bitscore', 'qcovs', 'qcovhsp')

    # Read in BLAST results
    markers_blast <- read.delim(path, sep = '\t', col.names = cols) %>%
        mutate(qseqid = as.character(qseqid)) %>% 
        mutate(orig_scaff_name = sseqid,
               sseqid = paste0(sprintf("%s%s_", sg, hap), str_extract(sseqid, pattern = 'Scaffold_\\d+'))) %>% 
        mutate(pop = marker_pop,
               sg = sg,
               hap = case_when(hap == 1 ~ 'One',
                               TRUE ~ 'Two'))
    
    return(markers_blast)
}

# Load and filter Minimap2 PAF file
load_filter_paf <- function(path, sg, hap, min_size){
    
    paf <- suppressMessages(read_delim(path, col_names = FALSE, delim = '\t')) %>%
        dplyr::select(X1:X12)
    
    names <- longestScaffolds_byLinkageGroup %>% 
        filter(subgenome == sg & haplotype == hap) %>% 
        pull(original_scaffold_name)

    paf_mod <- paf %>%
        filter(X12 == 60 & X11 >= min_size) %>%
        filter(X6 %in% names)
    
    return(paf_mod)
}

# Linkage group analysis

1. BLASTed Markers from Olsen _et al._ DG and SG mapping populations to all 4 Dovetail haplotypes (2 haplotypes per subgenome)
2. The goal is to see which chromosomes in the new assembly correspond to which linkage groups, and later which chromosomes these represent in the Griffiths TrR_v5 assembly.

In [5]:
# Load results from BLAST searches
genMap_df <- purrr::map_dfr(markers_blast_paths, load_marker_blast)

# Load dataframes with names of markers.
DG_names <- read.csv(DG_names_path, col.names = c('qseqid', 'name')) %>% mutate(pop = 'DG')
SG_names <- read.csv(SG_names_path, col.names = c('qseqid', 'name')) %>% mutate(pop = 'SG')
all_names <- bind_rows(SG_names, DG_names) %>% mutate(qseqid = as.character(qseqid))

# Load dataframes with genetic positions of markers. 
DG_genMap <- read.csv(DG_genMap_path, col.names = c('name', 'LG', 'cM')) %>% mutate(pop = 'DG')
SG_genMap <- read.csv(SG_genMap_path, col.names = c('name', 'LG', 'cM')) %>% mutate(pop = 'SG')
all_genMap <- bind_rows(SG_genMap, DG_genMap)

# Join all dataframes
genMap_df_mod <- genMap_df %>% 
    left_join(., all_names, by = c('qseqid','pop')) %>% 
    left_join(., all_genMap, by = c('name', 'pop'))
head(genMap_df_mod)

## Step 1: Map LGs to subgenomes 

Using the BLAST results, let's figure out which linkage groups correspond to the _T. occidental_ and _T. pallescens_ subgenomes.

In [6]:
BLASThits_LGxSG_plot <- genMap_df_mod %>% 
    group_by(name, sg, hap) %>% 
    
    # For each marker, subgenome, and haplotype, get only long markers with high % ident. 
    # Filter for best alignment 
    filter(pident >= 95 & length > 175) %>%
    filter(evalue == min(evalue)) %>%
    dplyr::select(LG, sseqid, pident, length, evalue, bitscore, sg, hap, name) %>%
    arrange(LG, name) %>%
    
    # Remove duplicates. Some top alignments have multiple start positions (but same cM)
    distinct() %>%
    ungroup() %>%

    # Get best alignment for each marker
    group_by(name) %>%
    filter(evalue == min(evalue)) %>%
    ungroup() %>%
    
    # Plot number of markers on each LG by subgenome
    group_by(LG, sg) %>% 
    summarise(n = n()) %>%
    ggplot(., aes(x = LG, y = n, fill = sg)) +
        geom_bar(stat = 'identity', position = 'dodge') +
        scale_x_continuous(breaks = seq(1, 16, 1)) +
        xlab('Linkage group') + ylab('Number of markers with top BLAST hit') +
        theme_classic() +
        theme(axis.title = element_text(size = 17),
              axis.text = element_text(size = 15),
              legend.position = 'top',
              legend.key.size = unit(1, 'cm'),
              legend.title = element_text(size = 17),
              legend.text = element_text(size=15))

BLASThits_LGxSG_plot
ggsave(filename = BLASThits_LGxSG_plot_path, plot = BLASThits_LGxSG_plot, 
       device = 'pdf', width = 8, height = 8, units = 'in', dpi = 300)

## Step 2: Map LGs to scaffolds in each subgenome

1. Generate set of filtered markers for use in figuring out which scaffolds correspond to each linkage group in each subgenome. 
2. Identify subgenome scaffolds in corresponding to each LG
3. Get longest scaffold for each linkage group. These will make up the chromosomes in our haploid reference assembly
    - One exception is LG 2, which contains Ac. Because the haplotype carrying a functional Ac is much shorter (~14 Mb), we will recombine the longer haplotype onto the shorter haplotype in a larger bit of unique sequence upstream of Ac.

### Step 2a: Filter markers

In [7]:
filtered_markers <- genMap_df_mod %>% 
    group_by(name, sg, hap) %>% 
    
    # For each marker, subgenome, and haplotype, get only long alignments with high % ident. 
    # Filter for best alignment 
    filter(pident >= 95 & length > 175) %>%
    filter(evalue == min(evalue)) %>%
    dplyr::select(LG, sseqid, pident, length, evalue, bitscore, sg, hap, name, slen) %>%
    arrange(LG, name) %>%
    
    # Remove duplicates. Some top alignments have multiple start positions (but same cM)
    distinct() %>%
    ungroup() %>%
    
    # Remove markers mapped to wrong subgenome
    # This is based on linkage group analysis above
    mutate(to_filter = case_when(LG %in% 1:8 & sg == 'occ' ~ 0,
                                 LG %in% 9:16 & sg == 'pall' ~ 0,
                                 TRUE ~ 1)) %>%
    filter(to_filter == 0) %>%
    mutate(scaff = str_extract(sseqid, pattern = 'Scaffold_\\d+'))

### Step 2b: Identify subgenome scaffolds corresponding to each LG

In [8]:
# Gereate DF with TrRv6 scaffolds corresponding to each LG. Add scaffold lengths
scaffolds_bySubgenome_byHaplotype <- filtered_markers %>%
    group_by(LG, sg, hap, scaff) %>%
    summarise(n = n()) %>%
    ungroup() %>%
    group_by(LG, sg, hap) %>%
    filter(n == max(n)) %>%
    arrange(LG, scaff) %>%
    left_join(., filtered_markers %>% dplyr::select(sg, hap, scaff, slen) %>% distinct(), 
              by = c('sg', 'hap', 'scaff'))

write_csv(scaffolds_bySubgenome_byHaplotype, scaffoldLengths_path)
head(scaffolds_bySubgenome_byHaplotype)

### Step 2c: Get longest scaffold for each linkage group

In [9]:
# Get original scaffold names
orig_scaff_names <- genMap_df_mod %>%
    mutate(scaff = str_extract(sseqid, pattern = 'Scaffold_\\d+')) %>%
    dplyr::select(sg, hap, scaff, orig_scaff_name) %>%
    distinct()
head(orig_scaff_names)

In [10]:
# Get longest scaffold for each linkage group. 
# These will make up the chromosomes in our genome
# The only exception will be LG 2, which contains Ac
longestScaffolds_byLinkageGroup <- scaffolds_bySubgenome_byHaplotype %>%
    ungroup() %>%
    group_by(LG) %>%

    # Keep shortest scaffold for LG 2 since this contains functional Ac locus
    # Use longest scaffold for all other LGs
    filter(case_when(LG == 2 ~ slen == min(slen),
                     TRUE ~ slen == max(slen))) %>%

    left_join(., orig_scaff_names, by = c('sg', 'hap', 'scaff')) %>%
    dplyr::select(-n, -scaff) %>%
    rename('subgenome' = 'sg',
           'haplotype' = 'hap',
           'length' = 'slen',
           'original_scaffold_name'= 'orig_scaff_name') %>%
    ungroup()

write_csv(longestScaffolds_byLinkageGroup, longestScaffolds_path)
head(longestScaffolds_byLinkageGroup)

## Step 3: Map v5 scaffolds to v6 chromosomes and rename

In [11]:
# Scaffolds for each LG
# See Previous assembly https://www.ncbi.nlm.nih.gov/data-hub/genome/GCA_005869975.1/
# See Olsen et al. paper https://nph.onlinelibrary.wiley.com/doi/10.1111/nph.17654
TrR_v5_LGs <- read_csv(TrR_v5_chromosomes_path, show_col_types = F)
head(TrR_v5_LGs)

In [14]:
# Create file with new TrRv6 chromosome names, original scaffold names and corresponding TrRv5 scaffold name
TrR_v6_chromosomes <- longestScaffolds_byLinkageGroup %>% 
    mutate(LG = LG) %>%
    left_join(., TrR_v5_LGs, by = 'LG') %>%
    arrange(TrR_v5_scaff_name) %>%
    mutate(subgenome = str_to_title(subgenome),
           chrom_num = sprintf("%02d",rep(row_number(), each=2, length.out = n())),
           TrR_v6_chromosome_name = paste0('Chr', chrom_num, '_', subgenome)) %>%
    dplyr::select(LG, TrR_v5_scaff_name, TrR_v6_chromosome_name, subgenome, haplotype, length, original_scaffold_name)
write_csv(TrR_v6_chromosomes, chrom_mapping_path)
TrR_v6_chromosomes

## Step 4: Look at orientation of v6 chromosomes against v5 scaffolds

1. Mapped TrR_v6 Dovetail chromosomes against scaffolds from Griffiths TrR_v6 scaffolds using `minimap2`
2. Goal is to have the orientation of TrR_v6 scffolds match the orientation of the existing TrR_v5 scaffolds
    - Will use correlation in `minimap2` alignment positions to get the chromosomes that need to be reverse complementd
    - Only using alignments with mapping quality of 60, length greater than 10Kb, and where both the query and subject are part of the same linkage group

In [15]:
# Load haplotype PAF files
# Warnings can be ignored since not using columns where they occur
min_size = 10000
occ1_paf_filtered <- load_filter_paf(occ1_paf_path, sg = 'occ', hap = 'One', min_size = min_size)
occ2_paf_filtered <- load_filter_paf(occ2_paf_path, sg = 'occ', hap = 'Two', min_size = min_size)
pall1_paf_filtered <- load_filter_paf(pall1_paf_path, sg = 'pall', hap = 'One', min_size = min_size)
pall2_paf_filtered <- load_filter_paf(pall2_paf_path, sg = 'pall', hap = 'Two', min_size = min_size)

In [16]:
# Combine PAF files
all_pafs <- bind_rows(occ1_paf_filtered, occ2_paf_filtered, pall1_paf_filtered, pall2_paf_filtered)

In [17]:
# Create dataframe with "mapping" column representing correct v5_v6 pairs
chromosome_mappings <- TrR_v6_chromosomes %>%
    mutate(mapping = paste(TrR_v5_scaff_name, original_scaffold_name, sep = '_')) %>%
    dplyr::select(original_scaffold_name, TrR_v6_chromosome_name, TrR_v6_chromosome_name, mapping)
chromosome_mappings

In [18]:
# Filter for v5_v6 mappings part of same LG. Plots alignment start positions by v6 Chromosome
chrom_orientations_plot <- all_pafs %>%
    rename("TrR_v5_scaff_name" = 'X1',
           "original_scaffold_name" = 'X6') %>%
    mutate(id = paste(TrR_v5_scaff_name, original_scaffold_name, sep = '_')) %>%
    left_join(., chromosome_mappings, by = 'original_scaffold_name') %>%
    filter(id == mapping) %>%
    ggplot(aes(x = X8, y = X3)) +
        geom_point(size = 2, color = 'black', fill = 'black') +
        geom_smooth(method = 'lm', color = 'blue', size = 1) +
        xlab('Dovetail start position') + ylab('Griffiths start position') +
        facet_wrap(~TrR_v6_chromosome_name) +
        theme_classic() +
        theme(axis.title = element_text(size = 17),
              axis.text = element_text(size = 15),
              axis.text.x = element_text(angle = 45, hjust = 1))

chrom_orientations_plot
ggsave(filename = chrom_orientations_plot_path, plot = chrom_orientations_plot, 
       device = 'pdf', width = 12, height = 12, units = 'in', dpi = 300)

In [42]:
models <- all_pafs %>%
    rename("TrR_v5_scaff_name" = 'X1',
           "original_scaffold_name" = 'X6') %>%
    mutate(id = paste(TrR_v5_scaff_name, original_scaffold_name, sep = '_')) %>%
    left_join(., chromosome_mappings, by = 'original_scaffold_name') %>%
    filter(id == mapping) %>%
    dplyr::select(TrR_v6_chromosome_name, X3, X8) %>% 
    nest_by(TrR_v6_chromosome_name) %>% 
    mutate(model = list(lm(X3 ~ X8, data = data))) %>%
    summarise(tidy(model)) %>% 
    filter(term == 'X8') %>% 
    filter(estimate < 0) %>% 
    dplyr::select(TrR_v6_chromosome_name)
models
write_csv(models, chrom_orientations_df_path)